In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import base64
import json
import logging
import os
import re
import smtplib
import sys
import time
import threading
from copy import deepcopy
from functools import partial
from itertools import chain, combinations, product
from numbers import Number
from pathlib import Path
from uuid import uuid4
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser
from scipy.optimize import minimize

from api.email import smtp_server, to_email_test, to_email_all, df_to_html, from_email
from api.risk.var import VaRPnl, get_db_from_clarion_positions, get_fut_trade_spec, get_var_ts, get_db_from_specs, var_ptf, get_clarion_fut_trade_spec
from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods, parse_swap_period
from gioutils.tools_clarion import get_clarion_positions
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
    SQL
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root,
    encode_df_as_image_tag
)

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)
rng = np.random.default_rng(7)

In [4]:
super_clarion(clarion)
runner = Runner(clarion)

In [5]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-04-19 10:57:56,121 - Authenticating...
2023-04-19 10:57:56,713 - Successfully authenticated


In [6]:
def send_live_var_email(
    var_close, 
    var_intraday, 
    var_new, 
    intraday_group, 
    trades_show,
    subject=f'Live VaR Reduction {pd.Timestamp.now(): %d%b%y %H:%M}'
):
    
    to_email = [
        'giovambattista.perciaccante@brevanhoward.com',
        'menashe.banit@brevanhoward.com'
    ]
    
    font_family = 'Helvetica, Arial, Sans-Serif'
    message = MIMEMultipart("alternative")
    message["Subject"] = subject
    message["From"] = from_email
    message["To"] = ','.join(to_email)

    file_name = 'tempVaR'
    dfs = [encode_df_as_image_tag(s, table_conversion='m') for s in [intraday_group, trades_show]]
    html_body = f"<h3 style=\"font-family:{font_family};\">Close VaR:{'&nbsp;'*40}{var_close: .2%}</h3>"
    html_body = html_body + f"<h3 style=\"font-family:{font_family};\">VaR with Intraday (below): {'&nbsp;'*13}{var_intraday: .2%}</h3>"
    html_body = html_body + dfs[0] + '<br>' * 3
    html_body = html_body + f"<h3 style=\"font-family:{font_family};\">VaR after the below trades:{'&nbsp;'*11}{var_new: .2%}</h3>"
    html_body = html_body + dfs[1] 
    html_ = MIMEText(html_body, "html")
    message.attach(html_)
    smtp = smtplib.SMTP(smtp_server)
    # smtp.set_debuglevel(1)
    smtp.sendmail(from_email, to_email, message.as_string())

### METRICS

In [7]:
clarion_metrics = dict(
    pv = clarion.metric('pv', 'PresentValue', 'Any'),
    local_pv = clarion.metric('local_pv', 'LocalPresentValue', 'Any'),
    # currency = clarion.metric('currency', 'PayCurrency', 'Any'),
    strike = clarion.metric('strike', 'Strike', 'Any'),
    notional = clarion.metric('notional', 'Notional', 'Any'),
    expiry = clarion.metric('expiry', 'ExpiryDate', 'Any'),
    maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'}),
    start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'}),
    fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates'),
    delta = clarion.metric('delta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    gamma = clarion.metric('gamma', 'Gamma', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    theta = clarion.metric('theta', 'OptionTheta', 'Rates', {'Period': '1'}),
    price_cents = clarion.metric('price_cents', 'Price', 'Rates', {'Unit': 'Cents'}),
    fwd_price_cents = clarion.metric('fwd_price_cents', 'ForwardPrice', 'Rates', {'Unit': 'Cents'}),
    price_bps = clarion.metric('price_bps', 'Price', 'Rates', {'Unit': 'BasisPoints'}),
    price_pct = clarion.metric('price_pct', 'Price', 'Equity', {'Unit': 'PercentLocal'}),
    price_unit = clarion.metric('price_unit', 'Price', 'Equity', {'Unit': 'Local'}),
    spot = clarion.metric('spot', 'Spot', 'Rates'),
    # vega = clarion.metric('vega', 'Vega', 'Rates', {'Period': 'Daily'}),
    # atm_vol_day = clarion.metric('atm_vol_day', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Daily'}),
    atm_vol_ann = clarion.metric('atm_vol_ann', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Annual'}),
    # strike_vol_day = clarion.metric('strike_vol_day', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'}),
    strike_vol_ann = clarion.metric('strike_vol_ann', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Annual'}),
    # carry_day = clarion.metric('carry_day', 'Carry', 'Credit', {'Horizon': '1d'}),
)

### VAR

In [78]:
tr_w = - wv.hist.sum(axis=1).nsmallest(1) / capital * 100

In [33]:
df_w = wv.hist.groupby(level=0, axis=1).sum()

In [79]:
tr_w.rename('tail').to_frame()

tail
returnDate       
2023-03-17 3.0902

In [50]:
(- df_w.loc[tr_w.index[0]] / capital * 100).rename('marginal_tail').to_frame().style.format(precision=2)

In [62]:
(df_w.loc[tr_w.index[0]]).rename('marginal_tail').to_frame()

marginal_tail
strategy                      
MU:ER VOL COND           2.73k
MU:EURCOND FLY           6.39k
MU:EURCOND FLY2       -593.51k
MU:EURVOL COND           9.09k
MU:EURVOL COND2       -367.39k
MU:EURVOL COND3         10.90k
MU:EURVOL RV             1.37m
MU:EURVOL RV2          344.70k
MU:FX_EURGBP           -0.2661
MU:GBPVOL RV           743.84k
MU:USD SHORT1            1.13m
MU:USD SHORT2          970.21k
MU:USD SHORT3           -5.66m
MU:USD SHORT4         -489.43k
MU:USDCOND FLY         537.59k
MU:USDVOL COND2        463.54k
MU:USDVOL LTD          489.85k
MU:USDVOL1              -5.47m
MU:USDVOLCUR RV        320.86k

In [67]:
tr_w - col.nsmallest(1)

returnDate
2023-03-17   5.47m
dtype: float64

In [74]:
res = {}
for index, col in df_w.items():
        res[index] = df_w.drop(index, axis=1).sum(axis=1).nsmallest(1).rename('marginal_tail').to_frame()

In [82]:
- (pd.concat(res) / capital * 100) - tr_w.squeeze()

marginal_tail
                returnDate               
MU:ER VOL COND  2023-03-17         0.0014
MU:EURCOND FLY  2023-03-17         0.0032
MU:EURCOND FLY2 2023-03-17        -0.2968
MU:EURVOL COND  2023-03-17         0.0045
MU:EURVOL COND2 2023-03-17        -0.1837
MU:EURVOL COND3 2023-03-17         0.0054
MU:EURVOL RV    2023-03-17         0.6874
MU:EURVOL RV2   2023-03-17         0.1724
MU:FX_EURGBP    2023-03-17        -0.0000
MU:GBPVOL RV    2023-03-17         0.3719
MU:USD SHORT1   2023-03-17         0.5627
MU:USD SHORT2   2023-03-17         0.4851
MU:USD SHORT3   2022-09-23        -1.4360
MU:USD SHORT4   2023-03-17        -0.2447
MU:USDCOND FLY  2023-03-17         0.2688
MU:USDVOL COND2 2023-03-17         0.2318
MU:USDVOL LTD   2023-03-17         0.2449
MU:USDVOL1      2022-09-23        -1.6282
MU:USDVOLCUR RV 2023-03-17         0.1604

In [81]:
tr_w.squeeze()

3.09016358085

In [70]:
pd.DataFrame(res).T

returnDate       2022-09-23  2023-03-17
MU:ER VOL COND          NaN      -6.18m
MU:EURCOND FLY          NaN      -6.19m
MU:EURCOND FLY2         NaN      -5.59m
MU:EURVOL COND          NaN      -6.19m
MU:EURVOL COND2         NaN      -5.81m
MU:EURVOL COND3         NaN      -6.19m
MU:EURVOL RV            NaN      -7.56m
MU:EURVOL RV2           NaN      -6.53m
MU:FX_EURGBP            NaN      -6.18m
MU:GBPVOL RV            NaN      -6.92m
MU:USD SHORT1           NaN      -7.31m
MU:USD SHORT2           NaN      -7.15m
MU:USD SHORT3        -3.31m         NaN
MU:USD SHORT4           NaN      -5.69m
MU:USDCOND FLY          NaN      -6.72m
MU:USDVOL COND2         NaN      -6.64m
MU:USDVOL LTD           NaN      -6.67m
MU:USDVOL1           -2.92m         NaN
MU:USDVOLCUR RV         NaN      -6.50m

In [61]:
col

returnDate
2018-04-20    247.49k
2018-04-27    401.40k
2018-05-04     23.22k
2018-05-11     20.03k
2018-05-18     11.26k
               ...   
2023-03-17     -5.47m
2023-03-24   -192.18k
2023-03-31      1.62m
2023-04-07   -585.24k
2023-04-14      1.01m
Name: MU:USDVOL1, Length: 261, dtype: float64

In [37]:
df_w.drop(index, axis=1).sum(axis=1).nsmallest(1).squeeze()

-6183054.785199999

In [38]:
tr.squeeze()

-6180327.1617

In [39]:
tr - df_w.drop(index, axis=1).sum(axis=1).nsmallest(1)

returnDate
2023-03-17   2.73k
dtype: float64

In [11]:
book = 'MU'
capital = 200e6
flag = 1.5 / 100 * capital

In [12]:
dv = VaRPnl()
wv = VaRPnl()
wv.get_data(
    offset="5y",
    exclude_cash=False,
    horizon='5d',
    book=book
)

C:\dev\projects\menashe\api\risk\var.py:172: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ).sum()


In [10]:
dv.get_data(book=book)
f"VaR: {- dv.var() / flag:.2%}"

C:\dev\projects\menashe\api\risk\var.py:172: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ).sum()


'VaR: 33.70%'

In [12]:
cob_ptf = dv.hist.sum(axis=1).rename('ptf')
# cob_ptf = dv.hist \
#     .drop([*dv.ref.query('tradeGroup == "OPT" and instrument == "USD CBT T 10Y"')['positionName']] , axis=1, level=1) \
#     .drop('MM:BOE FEB MAY', axis=1) \
#     .sum(axis=1).rename('ptf')

In [13]:
f"Cob VaR: {- cob_ptf.quantile(0.05, interpolation='weibull') / flag:.2%}"

'Cob VaR: 33.59%'

### UPDATE POSITIONS

In [ ]:
df = get_clarion_positions(clarion, book='MU-BAL')

2023-03-14 10:22:51,078 - Requesting data
2023-03-14 10:23:08,275 - Request complete
2023-03-14 10:23:08,296 - Response Count Summary - DFRAME:345


In [ ]:
intraday = df.query('isIntraday == True')

In [ ]:
intraday_group = intraday.groupby('Description')['Notional'].sum().replace(0, np.nan).dropna().to_frame()
rates_mask = intraday.loc[intraday['Category'] == 'rates', 'Description'].drop_duplicates().to_list()
intraday_rates = intraday_group.loc[intraday_group.index.isin(rates_mask)]
intraday_group

Notional
Description                                     
EUR EURIBOR 3M ICE JUN24                    -200
US LONG CBOT JUN23                           -25
USD SOFR 3M CME SEP23xSEP24 95.375 Put        1k
USD SOFR 3M CME SEP23xSEP24 96 Put           -2k
USD SOFR 3M CME SEP23xSEP24 96.625 Put        1k

### NEW TRADES VAR

In [ ]:
db = get_db_from_clarion_positions(intraday)
ts, res = get_var_ts(db)

In [ ]:
current_ptf = pd.concat([cob_ptf, ts], axis=1).sum(axis=1)

In [ ]:
f"Live VaR: {- current_ptf.quantile(0.05, interpolation='weibull') / flag:.2%}"

'Live VaR: 23.50%'

### MINIMIZE

In [20]:
max_size = 10e3

In [21]:
new_trades_list = [
    ('er', 'm23'),
    ('er', 'm24'),
    ('sfr', 'm24'),
    # ('sfi', 'u24'),
    # ('fv', 'm23'),
    # ('us', 'm23'),
]

new_trades_specs = [get_fut_trade_spec(underlying, contract) for underlying, contract in new_trades_list]
new_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in new_trades_list]

In [22]:
deltas = runner.run(metrics=[clarion_metrics['delta']], data_list=new_trades_clarion).loc[:, ['id', 'delta']]

2023-03-14 10:23:14,926 - Requesting data
2023-03-14 10:23:18,507 - Request complete
2023-03-14 10:23:18,514 - Response Count Summary - DFRAME:3


In [23]:
new_trades_db = get_db_from_specs(new_trades_specs)

In [24]:
new_trades_db

HTML(value='BH Handle')

0 SVMInstrument  Ccy        Generator MtyTnr PayoutType  Nominal Description  \
1          sfut  EUR     EUR LIFFE 3M    m23        pay        1       erm23   
2          sfut  EUR     EUR LIFFE 3M    m24        pay        1       erm24   
3          sfut  USD  USD SOFR CME 3M    m24        pay        1      sfrm24   

0  TradeId  
1        1  
2        2  
3        3

In [35]:
new_trades, res = get_var_ts(new_trades_db)

In [26]:
grouper = dv.ref.query('strategy == "MU:USDVOL1"')['positionName2']
irs_names = dv.ref.query('strategy == "MU:USDVOL1" and tradeGroup == "IRS"')['positionName2'].drop_duplicates()
irs_trades = dv.hist.loc[:, 'MU:USDVOL1'].groupby([*grouper], sort=False, axis=1).sum().loc[:, irs_names]

In [29]:
new_trades = pd.concat([new_trades, irs_trades], axis=1).dropna()

In [30]:
var_ptf(current_ptf, new_trades, wgt=np.full(shape=new_trades.shape[1], fill_value=1), flag=flag)

0.2547889921516747

In [36]:
# x0 = rng.normal(size=new_trades.shape[1])
x0 = np.full(shape=new_trades.shape[1], fill_value=-100)
x0 = rng.normal(size=new_trades.shape[1]) * 1000

scales = [*np.hstack([10  ** np.arange(0, np.log10(max_size) + 1), - 10  ** np.arange(0, np.log10(max_size) + 1)])]
max_iter = 5
max_try = max_iter * len(scales)
k= 0 

res_opt = None
for i in range(max_iter):
    for scale in scales:
        k = k + 1
        x0 = rng.normal(size=new_trades.shape[1]) * scale
        x0[x0 < -max_size] = -max_size
        x0[x0 > max_size] = max_size
        res_opt_ = minimize(
                lambda wgt: var_ptf(
                    current_ptf=current_ptf,
                    new_trades=new_trades,
                    wgt=wgt,
                    flag=flag,
                ),
                bounds=[(-max_size, max_size) for col in new_trades],
                x0=x0,
                # tol=0.001,
                method='Nelder-Mead', 
                # method='L-BFGS-B',
                # constraints=[
                #     {
                #         'type': 'ineq',
                #         'fun': lambda wgt: 0.3 + var(df=d1, wgt=wgt, periods=522, diff=True, scale=-1)
                #     },
                #     {
                #         'type': 'ineq',
                #         'fun': lambda wgt: 1 + var(df=d1, wgt=wgt, periods=522, agg=True, diff=True, scale=-1)
                #     },
                # ],
                # options={
                #     'maxiter': 10000000000000,
                # }
            )
        if res_opt is None:
            res_opt = res_opt_
        
        if res_opt_['fun'] < res_opt['fun']:
            res_opt = res_opt_
        
        print(f"{k} - {max_try} {res_opt['x']} {round(res_opt['fun'], 4)}", end='\r')
        

In [37]:
var_close = - cob_ptf.quantile(0.05, interpolation='weibull') / flag
var_intraday = - current_ptf.quantile(0.05, interpolation='weibull') / flag

trades = pd.Series(res_opt['x'].round(0), index=new_trades.columns.str.upper(), name='quantity')
var_new = res_opt['fun']
deltas_opt = pd.Series(deltas['delta'].to_numpy(), index=deltas['id'].str.upper())
delta_usd = (trades * deltas_opt).rename('delta$')

trades_show = pd.concat([trades, delta_usd], axis=1)
sum_ = trades_show.sum().rename('TOTAL')
sum_.iloc[0] = np.nan
sum_= sum_.to_frame().T
trades_show = pd.concat([trades_show, sum_])

In [38]:
display(w.HTML(f"<h3>Close VaR:{'&nbsp;'*40}{var_close: .2%}</h3>"))
display(w.HTML(f"<h3>VaR with Intraday (below): {'&nbsp;'*11}{var_intraday: .2%}</h3>"))
display(intraday_rates)
display(w.HTML(f"<h3>VaR after the below trades:{'&nbsp;'*11}{var_new: .2%}</h3>"))
display(trades_show)

HTML(value='<h3>Close VaR:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;…

HTML(value='<h3>VaR with Intraday (below): &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; …

Notional
Description                                     
EUR EURIBOR 3M ICE JUN24                    -200
US LONG CBOT JUN23                           -25
USD SOFR 3M CME SEP23xSEP24 95.375 Put        1k
USD SOFR 3M CME SEP23xSEP24 96 Put           -2k
USD SOFR 3M CME SEP23xSEP24 96.625 Put        1k

HTML(value='<h3>VaR after the below trades:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; …

quantity  delta$
ERM23       -952  25.47k
ERM24       -705  18.86k
SFRM24    -1.21k  30.25k
TOTAL        NaN  74.57k

In [ ]:
send_live_var_email(
    var_close, 
    var_intraday, 
    var_new, 
    intraday_rates, 
    trades_show
)

#### Random Search

In [ ]:
wgt_grid = np.arange(-max_size, max_size, 50)

In [ ]:
ress = {}
save_max = 1000
max_iter = 1e5
i = 0
min1 = 1e9
while True:
    i += 1
    wgt = rng.choice(wgt_grid, new_trades.shape[1]) if not isinstance(wgt_grid, dict) else np.asanyarray([rng.choice(wgt_grid[col])  for col in new_trades])
    var_ = var_ptf(current_ptf=current_ptf, new_trades=new_trades, wgt=wgt)
    ress[tuple(wgt)] = var_
    
    if len(ress) > save_max:
        ress.pop(max(ress, key=ress.get))
    
    min2 = min(ress.values())
#     if i == 200:
#         break
    if min2 < min1:
        min1 = min2
        wgt_ = wgt
        
    print(f"{i} - {wgt_} {round(min1, 4)}", end='\r')
    if i + 1 == max_iter:
        break

In [ ]:
res = pd.Series(ress, name='quantity').rename_axis(new_trades.columns).sort_values()

In [ ]:
var_close = - dv.var() / flag

var_intraday = - pd.concat([dv.hist.sum(axis=1).rename('ptf'), ts], axis=1).sum(axis=1).quantile(0.05, interpolation='weibull') / flag
trades = pd.Series(res.reset_index().iloc[0, :-1].to_numpy(), index=res.reset_index().columns[:-1].str.upper(), name='quantity').to_frame()
var_new = res.reset_index().iloc[0, -1]

In [ ]:
display(w.HTML(f"<h3>Close VaR:{'&nbsp;'*40}{var_close: .2%}</h3>"))
display(w.HTML(f"<h3>VaR with Intraday (below): {'&nbsp;'*11}{var_intraday: .2%}</h3>"))
# display(db)
display(w.HTML(f"<h3>VaR after the below trades:{'&nbsp;'*11}{var_new: .2%}</h3>"))
display(trades)

In [ ]:
def get_clarion_fut_trade_spec(underlying, contract):
    instrument = 'lfut' if underlying.lower() in lfut_list else 'sfut'
    gen = fut_gen_map.get(underlying.lower())

    return {
        'instrument': instrument,
        'ccy': gen_ccy_map.get(gen),
        'gen': gen,
        'mty_tnr': contract,
        'pay_type': 'pay',
        'notional': 1,
    }

In [ ]:
db_ = get_db_from_clarion_positions(df.query('Category == "rates"'))

In [ ]:
a = bh.bhTsFiRemoteRiskCalc(
    tradesdb=db_,
    startdate=today() - BDay() - parse_offset('2y'),
    enddate=today() - BDay(),
    pblevals=['PV'],
    irbtdbname=SQL.FITradePricer.db,
    irbtdbsrvname=SQL.FITradePricer.server,
    remoteservername=SQL.IRBTServer.server,
    remoteserverport=SQL.IRBTServer.port,
    timeout=120
)

In [ ]:
db_

In [ ]:
a.py.query("Status=='OK' and Metric == 'Rate'")

In [ ]:
new_trades.diff() 

In [ ]:
new_trades / new_trades.iloc[0]